# Import the required modules

In [66]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [2]:
os.chdir("E:\Data Science\Capstone Projects\Capstone Project_Xg boost")

# Read the dataset

In [3]:
dataset=pd.read_csv("Purchase.csv")

In [4]:
#dataset.head(2)
dataset.Field6.dtypes

dtype('O')

# Extract month, weekday,year from Original_Quote_Date column using date_time function in Pandas

In [5]:
dataset['Date']=pd.to_datetime(pd.Series(dataset['Original_Quote_Date']))

In [7]:
dataset=dataset.drop(["Original_Quote_Date" ], axis=1)
dataset.head(2)

,QuoteNumber,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,...,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64,Date
0,1,0,B,23,0.9403,0.0006,965,1.0200,N,17,...,9,-1,8,-1,18,-1,10,N,CA,2013-08-16
1,2,0,F,7,1.0006,0.0040,548,1.2433,N,6,...,10,-1,11,-1,17,-1,20,N,NJ,2014-04-22


In [8]:
dataset['weekday']=dataset['Date'].dt.dayofweek
dataset['weekyear']=dataset['Date'].dt.weekofyear
dataset['Year']=dataset['Date'].dt.year
dataset['Month']=dataset['Date'].dt.month

# Write a simple for loop to get the columns with datatypes as object and fill those columns missing values with Place Holder value.

In [9]:
cols=[]
for i in list(dataset.columns):
    if dataset[i].dtypes == 'O':
        cols.append(i)
        
    
    

In [10]:
cols

['Field6',
 'Field10',
 'Field12',
 'CoverageField8',
 'CoverageField9',
 'SalesField7',
 'PersonalField7',
 'PersonalField16',
 'PersonalField17',
 'PersonalField18',
 'PersonalField19',
 'PropertyField3',
 'PropertyField4',
 'PropertyField5',
 'PropertyField7',
 'PropertyField14',
 'PropertyField28',
 'PropertyField30',
 'PropertyField31',
 'PropertyField32',
 'PropertyField33',
 'PropertyField34',
 'PropertyField36',
 'PropertyField37',
 'PropertyField38',
 'GeographicField63',
 'GeographicField64']

In [11]:
dataset[cols]=dataset[cols].fillna(-1)

In [14]:
dataset[cols].isnull().sum().sum()

0

In [16]:
dataset.isnull().sum().sum()

324893

# Convert the Object columns to Numerical using Label Encoder(Not Dummies)

In [23]:
lbl=LabelEncoder()
for cols in dataset:
    dataset[cols]=lbl.fit_transform(list(dataset[cols].values))
    


# Train Test Split

In [28]:
y=dataset[['QuoteConversion_Flag']]

In [30]:
X=dataset.drop(["QuoteConversion_Flag"],axis=1)

In [33]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=0)

# Build the Model now

In [36]:
import xgboost
from xgboost import XGBClassifier

In [77]:
eval_set = [(X_test, y_test)]
model=xgb.XGBClassifier(n_estimators=100,eta=0.1,objective="binary:logistic",silent=0,eval_metric="error",
                        eval_set=eval_set,verbose=10,max_depth=10,
                        subsample=0.6,colsample_bytree=0.6,reg_lambda=0.6,gamma=5)

In [78]:
model.fit(X_train,y_train)

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, eta=0.1, eval_metric='error',
       eval_set=[(        QuoteNumber  Field6  Field7  Field8  Field9  Field10  Field11  \
179489       179489       6       9      30       0        1        7
227423       227423       6      22      31       0        1        7
222565       222565       4       7      17       1        2        9 ...             0
215329                     0
117790                     0

[78226 rows x 1 columns])],
       gamma=5, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=0.6, scale_pos_weight=1, seed=None,
       silent=0, subsample=0.6, verbose=10)

In [63]:
preds =model.predict(X_test)

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# best params:eta=0.1,n_estimators=100,max_depth=default(3),f1:0.74
# best params:eta=0.1,n_estimators=100,max_depth=10,f1:0.77
# bestparams:eta=0.1,n_estimators=100,max_depth=10,subsample=0.6,colsample_bytree=0.6,f1:0.78

In [64]:
print(classification_report(y_test,preds))

             precision    recall  f1-score   support

          0       0.93      0.98      0.96     63473
          1       0.90      0.69      0.78     14753

avg / total       0.92      0.93      0.92     78226



In [65]:
accuracy_score(preds,y_test)

0.9264694602817477

# Applying KFold and cross_val_scores

In [68]:
kfold=StratifiedKFold(n_splits=3,random_state=0)
result=cross_val_score(model,X_train,y_train,cv=kfold)

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to 

In [75]:
print(result.std())

print(result)

0.0010598605017703494
[0.92607202 0.92350679 0.92444364]
